In [31]:
import re
import os
import subprocess
source = "COGENT Public Looking Glass"
# destination = ["google.txt","apple.txt","bitcoin.txt","goldmansachs.txt","ntt.txt","spotify.txt","vodacom.txt","wikipedia.txt","oracle.txt"]
# destination = ["google.txt","apple.txt","bitcoin.txt","ntt.txt","spotify.txt"]
# destination = ["google.txt","alibaba.txt","bitcoin.txt","ntt.txt","spotify.txt","amazon.txt"]
destination = ["ntt.txt"]
def getdata(source,destination):
    file = open(source+"/"+destination,"r")
    data = {
        "AS NUMBER" : [],
        "IP ADDRESS": [],
        "IP RANGE" : [],
        "LOCATION" : [],
        "ORGANISATION": []
    }
    for line in file.readlines():

    
        s = re.findall('\[AS[0-9]+',line)
        
        if (len(s)>0 and s[0][1] != "*"):
            
            ips = re.findall('[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+',line)

            statement = "./IPGeoLocation/ipgeolocation.py -t {} ".format(ips[0])
            proc = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc.communicate()
            out = out.decode()
            city = re.findall('City:[^\n]*\\n',out)
            city = re.findall('[A-Z][a-zA-Z ]*',city[0])
            country= re.findall('Country:[^\n]*\\n',out)
            country = re.findall('[A-Z][a-zA-Z ]*',country[0])
            org= re.findall('Organization:[^\n]*\\n',out)
            org = re.findall('[A-Z][a-zA-Z ]*',org[0])
            if (len(org) >=2):
                org = org[1]
            else:
                org = ""
            if (len(city) >=2):
                city = city[1]
            else:
                city = ""
            if (len(country) >=2):
                country = country[1]
            else:
                country = ""
            
            loc = city+'/'+country
            statement = "whois -a {}".format(ips[0])
            proc2 = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc2.communicate()
            out = out.decode()
            iprange = re.findall('inetnum:[ \t]*[-0-9\. ]*|NetRange:[ \t]*[-0-9\. ]*',out)
            if (len(iprange) > 0):
                iprange = iprange[0].split(":")[1].lstrip()
            else:
                iprange = ""
                
            
            data["IP RANGE"].append(iprange)
            data["AS NUMBER"].append(s[0][1:])
            data["IP ADDRESS"].append(ips[0])
            data["ORGANISATION"].append(org)
            data["LOCATION"].append(loc)
            
    file.close()
    return data


In [32]:
# getdata(source,destination[0])
destination_dict = {}
for d in destination:
    print(d)
    destination_dict[d] = getdata(source,d)
destination_dict

ntt.txt


{'ntt.txt': {'AS NUMBER': ['AS174',
   'AS2914',
   'AS2914',
   'AS2914',
   'AS2914',
   'AS2914',
   'AS2914',
   'AS2914',
   'AS2914',
   'AS2914',
   'AS2914',
   'AS4713',
   'AS4713'],
  'IP ADDRESS': ['130.117.254.105',
   '129.250.6.41',
   '129.250.6.1',
   '129.250.2.111',
   '129.250.6.147',
   '129.250.6.17',
   '129.250.2.27',
   '129.250.5.17',
   '129.250.2.119',
   '129.250.2.40',
   '61.200.80.74',
   '153.149.219.145',
   '61.113.104.9'],
  'IP RANGE': ['',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '153.128.0.0 - 153.253.255.255',
   ''],
  'LOCATION': ['Amsterdam/Netherlands',
   'Frankfurt am Main/Germany',
   'Frankfurt am Main/Germany',
   'Ashburn/United States',
   'Newark/United States',
   'Newark/United States',
   'Chicago/United States',
   'San Jose/United States',
   'Osaka/Japan',
   'Osaka/Japan',
   'Chiyoda/Japan',
   'Chiyoda/Japan',
   'Osaka/Japan'],
  'ORGANISATION': ['PSINet',
   'NTT America',
   'NTT America',
 

In [29]:
destination_dict

{'bitcoin.txt': {'AS NUMBER': ['AS174', 'AS174', 'AS13335'],
  'IP ADDRESS': ['130.117.254.105', '154.25.7.225', '104.22.68.176'],
  'IP RANGE': ['', '', '104.16.0.0 - 104.31.255.255'],
  'LOCATION': ['Amsterdam/Netherlands',
   'Cleveland/United States',
   'Toronto/Canada'],
  'ORGANISATION': ['PSINet', 'PSINet', 'Cloudflare']}}

In [30]:
import pandas as pd

for d in destination:
    df = pd.DataFrame(destination_dict[d])
    df.to_excel(source +"/"+"structured" + "/" + d[:-3]+"xlsx")